In [16]:
from src.explainer.explainer import ArgumentativeExplainer
from src.explainer.framework import ArgumentationFramework

from src.explainer.adjective import BooleanAdjective, PointerAdjective, ComparisonAdjective, MaxRankAdjective, MinRankAdjective
from src.explainer.explanation import Possession, Assumption, PossessionCondition, ConditionalExplanation, CompositeExplanation

In [17]:
# Example usage:
class MinMaxNode:
    def __init__(self, id, *, score=None, maximizing_player_turn=True, children=None, score_child=None):
        
        self.id = id

        if score:
            self.score = score
        elif score_child:
            self.score_child = score_child
            self.score = score_child.score
        else:
            raise ValueError("Provide score or score_child.")

        self.children = children or []
        self.parent = None
        self.maximizing_player_turn = maximizing_player_turn
        
        self.is_leaf = True
        if len(self.children) > 0:
            self.is_leaf = False
            for child in children:
                child.parent=self
                child.maximizing_player_turn = not self.maximizing_player_turn
    
    def __str__(self):
        return self.id

# Create a simple game tree
leaf11 = MinMaxNode('leaf11', score=3)
leaf12 = MinMaxNode('leaf12', score=4)
leaf21 = MinMaxNode('leaf21', score=8)
leaf22 = MinMaxNode('leaf22', score=2)
child1 = MinMaxNode('child1', children=[leaf11, leaf12], score_child=leaf11)
child2 = MinMaxNode('child2', children=[leaf21, leaf22], score_child=leaf22)
root = MinMaxNode('root', maximizing_player_turn=True, children=[child1, child2], score_child=child1)

In [18]:
knowledgebase = ArgumentationFramework()

knowledgebase.add_adjective( 
    BooleanAdjective("leaf")
)

knowledgebase.add_adjective( 
    PointerAdjective("score",

    explanation = ConditionalExplanation(
        condition = PossessionCondition("leaf"),
        true_explanation = Assumption("Leaf nodes have scores from the evaluation function"),
        false_explanation = CompositeExplanation(
            Assumption("Internal nodes have scores from children"),
            Possession("backtracing child"))
    ))
)

knowledgebase.add_adjective(
    BooleanAdjective("opponent player turn")
)

knowledgebase.add_adjective( 
    PointerAdjective("backtracing child",

    explanation = ConditionalExplanation(
        condition = PossessionCondition("opponent player turn"),
        true_explanation = CompositeExplanation(
            Assumption("We assume the opponent will do their best move."),
            Possession("backtracing child", "worst")),
        false_explanation = CompositeExplanation(
            Assumption("On our turn we take the maximum rated move."),
            Possession("backtracing child", "best"))
    ))
)

knowledgebase.add_adjective(
    ComparisonAdjective("better", "score", ">")
)

knowledgebase.add_adjective( 
    PointerAdjective("siblings")
)

knowledgebase.add_adjective(
    MaxRankAdjective("best", "better", "siblings")
)
knowledgebase.add_adjective(
    MinRankAdjective("worst", "better", "siblings")
)

In [19]:
#print(knowledgebase)

In [20]:
explainer = ArgumentativeExplainer(knowledgebase)
explainer.set_getter("siblings", lambda node: [sibling for sibling in node.parent.children if sibling is not node])
explainer.set_getter("leaf", lambda node: node.is_leaf)
explainer.set_getter("backtracing child", lambda node: node.score_child)
explainer.set_getter("score", lambda node: node.score)
explainer.set_getter("opponent player turn", lambda node: not node.maximizing_player_turn)
# alternative getter for backtracing child:
# lambda node: min(node.children, key=lambda child: child.score),

explainer.set_tree_search_motivation(lambda root: root.children, "best")

Try

In [21]:
# Generate explanations
print(explainer.explain_adjective(child1, "best"))


[(assumption) By definition a node is best if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) By definition of leaf → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)] → leaf11 is worst] → child1 has backtracing child = leaf11] → child1 has score = 3]
 ∧ [[(assumption) By defi

In [22]:
print(explainer.explain_adjective(child1, "worst"))

[(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) By definition of leaf → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf11 is better than leaf12)] → leaf11 is worst] → child1 has backtracing child = leaf11] → child1 has score = 3]
 ∧ [[(assumption) By def

In [23]:
print(explainer.explain_adjective(child2, "best"))

[(assumption) By definition a node is best if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → ¬(child2 is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → child2 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → leaf22 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf22 has score = 2]
 ∧ [[(assumption) By definition of leaf → leaf21 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf21 has score = 8]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2 score → ¬(leaf22 is better than leaf21)] → leaf22 is worst] → child2 has backtracing child = leaf22] → child2 has score = 2]
 ∧ [[(assumption) By defi

In [24]:
print(explainer.explain_adjective(root, "backtracing child"))

[[(assumption) By definition of opponent player turn → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move.
 ∧ [(assumption) By definition a node is best if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) By definition of leaf → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf12 has score = 4]
 ∧ [(assumption) By definition, node1 is "better" than node2 if node1 score > node2

In [25]:
print(explainer.explain_adjective(root, "score"))

[[(assumption) By definition of leaf → ¬(root is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → ¬(root is opponent player turn)]
 ∧ (assumption) On our turn we take the maximum rated move.
 ∧ [(assumption) By definition a node is best if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → ¬(child1 is leaf)]
 ∧ (assumption) Internal nodes have scores from children
 ∧ [[(assumption) By definition of opponent player turn → child1 is opponent player turn]
 ∧ (assumption) We assume the opponent will do their best move.
 ∧ [(assumption) By definition a node is worst if it's better compared to all nodes among siblings
 ∧ [[(assumption) By definition of leaf → leaf11 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation function → leaf11 has score = 3]
 ∧ [[(assumption) By definition of leaf → leaf12 is leaf]
 ∧ (assumption) Leaf nodes have scores from the evaluation 

Track down that Not before the "Considering your definition of leaf"

In [26]:
print(explainer.query_explanation(root, "Why is child 1 maxoptimal?"))

None
